In [5]:
import spotipy
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [4]:
load_dotenv()

True

In [9]:
scope = ['ugc-image-upload'
        ,'user-read-recently-played'
        ,'user-top-read'
        ,'user-read-playback-position'
        ,'user-read-playback-state'
        ,'user-modify-playback-state'
        ,'user-read-currently-playing'
        ,'app-remote-control'
        ,'streaming'
        ,'playlist-modify-public'
        ,'playlist-modify-private'
        ,'playlist-read-private'
        ,'playlist-read-collaborative'
        ,'user-follow-modify'
        ,'user-follow-read'
        ,'user-library-modify'
        ,'user-library-read'
        ,'user-read-email'
        ,'user-read-private'
        ]


auth_manager = spotipy.oauth2.SpotifyOAuth(scope=scope,
                                            show_dialog=True)

# auth_manager.get_access_token(request.args.get("code"))
auth_url = auth_manager.get_authorize_url()
spotify = spotipy.Spotify(auth_manager=auth_manager)


In [292]:
artists_items = spotify.current_user_top_artists(limit=100, time_range='long_term')['items']

In [116]:
temp = pd.DataFrame.from_dict(artists_items)

In [113]:
def clean_top_artists(items_df):
    '''
        
    '''

    items_df['external_urls'] = temp['external_urls'].map(lambda x: x['spotify'])
    items_df['images'] = temp['images'].map(lambda x: x[0]['url'])
    items_df['followers'] = temp['followers'].map(lambda x: x['total'])

    return items_df

In [169]:
def clean_top_tracks(tracks_df):
    '''
        
    '''
    tracks_df['album_name'] = temp['album'].map(lambda x:x['name'])
    tracks_df['album_art'] = temp['album'].map(lambda x:x['images'][0]['url'])
    tracks_df['album_href'] = temp['album'].map(lambda x:x['href'])
    tracks_df['album_release_date'] = temp['album'].map(lambda x:x['release_date'])

    tracks_df['artist_names'] = temp['artists'].map(lambda x: ', '.join([a['name'] for a in x]))

    drop_cols = ['album', 'artists', 'available_markets', 'disc_number', 'external_ids', 'is_local', 'external_urls', 'track_number']
    tracks_df = tracks_df.drop(drop_cols, axis=1)

    return tracks_df

In [371]:
def clean_playlist(user_playlists, playlist):
    '''

    '''

    filter_playlist = [i for i in spotify.current_user_playlists()['items'] if i['name']==playlist_name][0]
    filter_id = filter_playlist['id']
    playlist_filter_id = spotify.playlist(filter_id)
    playlist_tracks = playlist_filter_id['tracks']
    list_tracks = [playlist_tracks['items'][i]['track'] for i in range(filter_playlist['tracks']['total'])]
    temp = pd.DataFrame.from_dict(list_tracks)
    temp_df = clean_top_tracks(temp)

    return temp_df

In [392]:
def audio_playlist_features(playlist_df):
    '''

    '''
    return pd.DataFrame.from_dict([spotify.audio_features(playlist_df['id'][i])[0] for i in range(len(playlist_df))])
     

In [192]:
term = 'short'

In [193]:
track_items =spotify.current_user_top_tracks(limit=100, time_range=f'{term}_term')['items']

In [194]:
temp = pd.DataFrame.from_dict(track_items)

In [195]:
temp_df = clean_top_tracks(temp)

In [196]:
len(temp_df)

50

In [197]:
temp_df.to_csv(f'top_tracks_{term}.csv')

In [372]:
user_playlists = [i['name'] for i in spotify.current_user_playlists()['items']]
user_playlists

['229Jams',
 'Lorem',
 'RapCaviar',
 'Discover Weekly',
 'Hindi Songs - Mega Hit Mix - Bollywood songs - All time top 1000 songs',
 'Coffee Table Jazz',
 'Body Radio',
 'Classical Upbeat Piano',
 'Best of the Decade For You',
 'Your Top Songs 2019',
 'Hot Rhythmic',
 'lofi hip hop music - beats to relax/study to',
 'Late Night Jazz',
 'Liked from Radio',
 'Kimi No Nawa (Your Name) OST | RADWIMPS 💫 ',
 'Moood🤙',
 'All Out 10s',
 'Hamilton!']

In [373]:
playlist_name = 'Kimi No Nawa (Your Name) OST | RADWIMPS 💫 '

In [375]:
playlist_df =  clean_playlist(user_playlists, playlist_name)

In [394]:
playlist_df.head()

,duration_ms,episode,explicit,href,id,name,popularity,preview_url,track,type,uri,album_name,album_art,album_href,album_release_date,artist_names
0,534120,False,False,https://api.spotify.com/v1/tracks/0b4a1iklB8w8...,0b4a1iklB8w8gsE38nzyEx,Sparkle - Movie Version,0,None,True,track,spotify:track:0b4a1iklB8w8gsE38nzyEx,Your Name.,https://i.scdn.co/image/ab67616d0000b273222e0f...,https://api.spotify.com/v1/albums/3qNUgzsX6eVS...,2016-08-24,RADWIMPS
1,185229,False,False,https://api.spotify.com/v1/tracks/0BWtbkbX1hzJ...,0BWtbkbX1hzJf4OgeEz2Tn,"Kataware Doki (From ""Kimi No Nawa"") - Instrume...",36,https://p.scdn.co/mp3-preview/6011cd0208e3d140...,True,track,spotify:track:0BWtbkbX1hzJf4OgeEz2Tn,"Kataware Doki (From ""Kimi No Nawa"") [Instrumen...",https://i.scdn.co/image/ab67616d0000b273572452...,https://api.spotify.com/v1/albums/7HVdRlt7eQpI...,2020-07-25,Edward Ong
2,130266,False,False,https://api.spotify.com/v1/tracks/39ZbD2BuYmJu...,39ZbD2BuYmJudCHFankou3,Dream lantern - English ver.,35,None,True,track,spotify:track:39ZbD2BuYmJudCHFankou3,Your name. (deluxe edition / Original Motion P...,https://i.scdn.co/image/ab67616d0000b273036b85...,https://api.spotify.com/v1/albums/5BWM0Xyb6p59...,2018-03-10,RADWIMPS
3,201601,False,False,https://api.spotify.com/v1/tracks/45wVHKdU30J7...,45wVHKdU30J7YGd7xQu2zU,"Nandemonaiya (From ""Kimi No Nawa"") - Instrumental",35,https://p.scdn.co/mp3-preview/6d41cb5aeed63e7c...,True,track,spotify:track:45wVHKdU30J7YGd7xQu2zU,"Nandemonaiya (From ""Kimi No Nawa"") [Instrumental]",https://i.scdn.co/image/ab67616d0000b2737734c1...,https://api.spotify.com/v1/albums/2RHMcsLWdf3G...,2020-07-08,Edward Ong
4,67266,False,False,https://api.spotify.com/v1/tracks/5lAJ5nWaJQgl...,5lAJ5nWaJQglDcVvcCCBSn,Mitsuha No Tsugaku,0,None,True,track,spotify:track:5lAJ5nWaJQglDcVvcCCBSn,Your Name.,https://i.scdn.co/image/ab67616d0000b273222e0f...,https://api.spotify.com/v1/albums/3qNUgzsX6eVS...,2016-08-24,RADWIMPS


In [395]:
playlist_df.to_csv('playlist.csv')

In [387]:
# def get_features(sp, track_id):
#     '''
#     Returns a dictionary of features features that is used in KNN model
    
#     artist: Artist for which features are needed - str
#     sp: Spotify Token - spotipy.client.Spotify
#     features: List of features to return - List
#     '''    
#     d = dict()
#     audio_features = sp.audio_features(track_id)[0]
#     for feats in features:
#         if feats not in d.keys():
#             if feats in audio_features:
#                 d[feats] = audio_features[feats]
#     return d

In [396]:
playlist_features = audio_playlist_features(playlist_df)

In [397]:
playlist_features.to_csv('playlist_features.csv')